<h2>LSTM Wandb Stocks</h2>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [2]:
#@Libraries for Stocks
from yahoo_fin.stock_info import get_data
from datetime import datetime
from matplotlib import rcParams

#@Libraries Basics
from time import time
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
import pandas as pd
import os
import math
import random

#@Libraries Wandb 
import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

#@Libraries Normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

#@ Libraries Tensor Flow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,TensorBoard
from tensorflow.keras import callbacks
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner
from keras_tuner.tuners import RandomSearch,BayesianOptimization
from keras.utils import plot_model

print("Libraries loaded!!")

Libraries loaded!!


In [3]:
wandb.login()

wandb: Currently logged in as: sal-souzasa. Use `wandb login --relogin` to force relogin


True

<h3>Setup Jupyter Lab</h3>

In [4]:
#@ Settings Notebook
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Suprime as mensagens do tensor flow

In [5]:
START_DATE = dt.datetime(2020,1,1)

END_DATE = dt.datetime(2023,1,22)

dataset = get_data('BTC-USD', 
                   start_date=START_DATE, 
                   end_date=END_DATE, 
                   index_as_date = True, 
                   interval="1d")

dataset = dataset.dropna()

dataset_total = dataset.iloc[:,3:4].dropna()

In [6]:
print(f"O comprimento total do dataset é: {len(dataset_total)}")

O comprimento total do dataset é: 1848


<h3>Building dataset</h3>

In [7]:
dataset_values = dataset_total.values 

training_dataset_len = math.ceil(len(dataset_values)* 0.8) # 80% training

mms = MinMaxScaler(feature_range=(0,1)) # range for to scale (0,1)

scaled_dataset = mms.fit_transform(dataset_values.reshape(-1,1))  # normalized dataset and reshape(-1,1)

training_dataset = scaled_dataset[0: training_dataset_len, :]

In [8]:
training_dataset_len

1479

In [9]:
qt_days = 5

In [10]:
X_train = []
y_train = []

for i in range(qt_days, len(training_dataset)):
    X_train.append(training_dataset[i-qt_days:i, 0])
    y_train.append(training_dataset[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [11]:
test_dataset = scaled_dataset[training_dataset_len-qt_days: , : ]

print(f"O comprimento do teste é: {len(test_dataset)}")

x_test = []

y_test = dataset_values[training_dataset_len:]

for i in range(qt_days, len(test_dataset)):
    x_test.append(test_dataset[i-qt_days:i, 0])

x_test = np.array(x_test)

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

print(f"Dados de teste para o modelo: {x_test.shape}")

O comprimento do teste é: 374
Dados de teste para o modelo: (369, 5, 1)


In [12]:
# # Visualizando os dados x(t) e y(t+1) que vão entrar no modelo

# for entrada, saida in zip(X_train[0:2], y_train[0:2]):
#     print(f"\n## Aqui estamos trabalhando com base de prediçao de {qt_days} dias ##\n")
#     print(entrada, "<------>", saida)

In [13]:
# Definindo as entradas do modelo LSTM 

quantidade_dias = X_train.shape[1]

quantidade_features = X_train.shape[2]

input_shape = (quantidade_dias, quantidade_features)

print(f"Input Shape: {input_shape}")
print(f"Quantidade de dias base de treinamento: {quantidade_dias}")
print(f"Quantidade de features: {quantidade_features}")

Input Shape: (5, 1)
Quantidade de dias base de treinamento: 5
Quantidade de features: 1


<h3>Summary of data</h3>

In [14]:
print(f"Shape Train X: {X_train.shape}")
print(f"Shape Train Y: {y_train.shape}")
print(f"Shape Test X: {x_test.shape}")
print(f"Shape Test Y: {y_test.shape}")
print(f"Length: {training_dataset_len}")
print(f"Number of days (timePeriod): {qt_days}")

Shape Train X: (1474, 5, 1)
Shape Train Y: (1474,)
Shape Test X: (369, 5, 1)
Shape Test Y: (369, 1)
Length: 1479
Number of days (timePeriod): 5


<h3>Hyperparameter tuning using Wandb</h3>

<h3>Model</h3>

In [15]:
input_shape = (X_train.shape[1],1)

input_shape

(5, 1)

In [16]:
def LSTM(dropout=0.2):
    return keras.Sequential(
        [
        keras.Input(shape=input_shape),   
        layers.LSTM(50, return_sequences=True),
        layers.Dropout(dropout),
        layers.LSTM(50, return_sequences=True),
        layers.Dropout(dropout),
        layers.LSTM(50),
        layers.Dropout(dropout),
        layers.Dense(1, activation='tanh')
        ]
)

In [17]:
model = LSTM()

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 50)             10400     
                                                                 
 dropout (Dropout)           (None, 5, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 5, 50)             20200     
                                                                 
 dropout_1 (Dropout)         (None, 5, 50)             0         
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 5

In [19]:
def get_optimizer(lr=1e-3, optimizer="adam"):
    "Select optmizer between adam and sgd with momentum"
    if optimizer.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=lr)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)

In [20]:
def train(model, batch_size=32, epochs=3, lr=1e-3, optimizer='adam', log_freq=10):  
    
    # Compile model like you usually do.
    tf.keras.backend.clear_session()
    model.compile(loss="mean_squared_error", 
                  optimizer=get_optimizer(lr, optimizer), 
                  metrics=["mse"])

    # callback setup
    wandb_callbacks = [
        WandbMetricsLogger(log_freq=log_freq),
        WandbModelCheckpoint(filepath="my_model_{epoch:02d}")
    ]

    model.fit(X_train, 
              y_train, 
              batch_size=batch_size, 
              epochs=epochs, 
              validation_data=(x_test, y_test), 
              callbacks=wandb_callbacks)
    
    predictions = model.predict(x_test)
    predictions = mms.inverse_transform(predictions)
    rmse = np.sqrt(np.mean(predictions - y_test)**2)
    print(f"RMSE: {rmse}")
    mape = (mean_absolute_percentage_error(y_test, predictions)*100)
    print(f"MAPE: {mape}")
    wandb.log({'RSME': rmse, 
               'MAPE' : mape})

<h3>Sweep</h3>

In [21]:
sweep_config = {
    'method': 'bayes'
    }

In [22]:
metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [23]:

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'dropout': {
          'values': [0.1,0.2,0.3]
        },
    }

sweep_config['parameters'] = parameters_dict

In [24]:
parameters_dict.update({
    'epochs': {
        'value': 100 }
    })

In [25]:
import math

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.001,
        'max': 0.1
      },
    'batch_size': {
        'value': 64
      }
    })
     

In [26]:
import pprint

pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'value': 64},
                'dropout': {'values': [0.1, 0.2, 0.3]},
                'epochs': {'value': 100},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0.001},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [27]:
def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration, if any
        wandb.config.architecture_name = "LSTM"
        wandb.config.dataset_name = "BTC"
        wandb.config.timeperiod = "5d"

        # initialize model
        model = LSTM(wandb.config.dropout)

        train(model, 
              wandb.config.batch_size, 
              wandb.config.epochs,
              wandb.config.learning_rate,
              wandb.config.optimizer)

In [28]:
sweep_id = wandb.sweep(sweep_config, project="experiment-c")

Create sweep with ID: 5csk477r
Sweep URL: https://wandb.ai/sal-souzasa/experiment-c/sweeps/5csk477r


In [ ]:
wandb.agent(sweep_id, function=sweep_train, count=6)

wandb: Agent Starting Run: z3ek42t7 with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.1
wandb: 	epochs: 100
wandb: 	learning_rate: 0.08064728828059882
wandb: 	optimizer: adam


Epoch 1/100
24/24 [==============================] - ETA: 0s - loss: 0.2704 - mse: 0.2704

INFO:tensorflow:Assets written to: my_model_01\assets


INFO:tensorflow:Assets written to: my_model_01\assets
wandb: Adding directory to artifact (.\my_model_01)... Done. 0.1s


24/24 [==============================] - 63s 2s/step - loss: 0.2704 - mse: 0.2704 - val_loss: 818312448.0000 - val_mse: 818312448.0000
Epoch 2/100
23/24 [===========================>..] - ETA: 0s - loss: 0.0976 - mse: 0.0976

...

INFO:tensorflow:Assets written to: my_model_99\assets
wandb: Adding directory to artifact (.\my_model_99)... Done. 0.0s


24/24 [==============================] - 50s 2s/step - loss: 0.0011 - mse: 0.0011 - val_loss: 818291456.0000 - val_mse: 818291456.0000
Epoch 100/100
22/24 [==========================>...] - ETA: 0s - loss: 0.0012 - mse: 0.0012

INFO:tensorflow:Assets written to: my_model_100\assets


INFO:tensorflow:Assets written to: my_model_100\assets
wandb: Adding directory to artifact (.\my_model_100)... Done. 0.0s


12/12 [==============================] - 3s 7ms/step
RMSE: 866.9391434832318
MAPE: 6.543184175907182


In [ ]:
print("done!!!")